In [1]:
#
# This notebook has been written to accompany the version of the paper "Explicit
# Formulas for Permutation Pattern Character Polynomials" from August 15, 2023.
# In the code below, we adopt the following conventions:
# - Many of the functions defined here have counterparts in the paper. In such
#   a case, we name the function after the letter or symbol used in the
#   corresponding definition from the paper. We prefix all such functions with
#   `cp_` to avoid any name conflicts.
# - In order to avoid name conflicts with local variables, we prefix all
#   polynomial ring variables with `var_` and all power series ring variables
#   with `var_`.
# - We use PascalCase for functions returning sets or lists.
#

In [2]:
################################################################################
# Utility functions.                                                           #
################################################################################

#
# Returns the collection of sublists of `[1, ..., n]` of length `k`.
#
def Sublists(n, k):
  return Combinations(IntegerRange(1, n+1), k)

#
# Given a finite sequence (e.g., a list or an iterator), returns true if the
# sequence is strictly increasing.
#
def is_increasing(seq):
  # This code works when `seq` is any iterator.
  prev = None
  for next in seq:
    if prev is not None and next <= prev:
      return False
    prev = next
  return True

#
# Computes the bifactorial of an integer.
#
def bifactorial(n):
  return Integer(n).multifactorial(2)

#
# Returns true iff all of the coefficients of the given polynomial are
# nonnegative.
#
def has_nonnegative_coefficients(poly):
  return all(poly.monomial_coefficient(mon) >= 0 for mon in poly.monomials())

#
# Returns a list of real roots of a polynomial with rational coefficients,
# either as exact algebraic real numbers or using the specfiied precision.
#
def real_roots(poly, prec=None):
  roots = []
  for (p, e) in factor(poly):
    K.<x> = NumberField(p)
    embeddings = K.embeddings(AA) if prec is None else K.real_embeddings(prec)
    roots.extend(e * [sigma(x) for sigma in embeddings])
  return sorted(roots)

In [3]:
################################################################################
# Direct computation of $a^\lambda(n, k)$.                                     #
################################################################################

#
# Returns the symmetric group character corresponding to a given partition,
# denoted by $\chi^\lambda$ in the paper. Since computing this character can be
# an expensive operation, previously computed characters are cached for later
# use.
#
def cp_chi(lam):
  global _cp_chi_cache
  # We allow `lam` to be specified as a partition or a list.
  lam = Partition(lam)
  # If the desired character has already computed, return it.
  chi = _cp_chi_cache.get(lam)
  if chi is not None:
    return chi
  # Otherwise, compute the character, cache it, and return it.
  chi = SymmetricGroupRepresentation(lam).to_character()
  _cp_chi_cache[lam] = chi
  return chi
_cp_chi_cache = {}

#
# Returns the number of length `k` increasing subsequences of a permutation
# `pi`, corresponding to $N_k(\pi)$ in the paper.
#
def cp_N(k, pi):
  n = len(pi.domain())
  return Integer(
    sum(1 for S in Sublists(n, k) if is_increasing((pi(i) for i in S)))
  )

#
# Directly computes the value $a^\lambda(n, k)$ from the paper.
#
def cp_a(lam, n, k):
  # We allow `lam` to be specified as a partition or a list.
  lam = Partition(lam)
  # The function `Partition.t_completion` corresponds to the notation
  # `\lambda[n]` from the paper. In particular, it will automatically raise an
  # exception if `n < lam.size() + lam.get_part(0)`.
  lam_completion = lam.t_completion(n)
  chi = cp_chi(lam_completion)
  return sum(chi(pi) * cp_N(k, pi) for pi in SymmetricGroup(n)) / factorial(n)

In [4]:
################################################################################
# Functions on partitions.                                                     #
################################################################################

#
# Returns the number of parts of size `j` in a partition, denoted by
# `m_j(\lambda)` in the paper.
#
def _partition_multiplicity(self, j):
  return Integer(sum(1 for part in self if part == j))
Partition.multiplicity = _partition_multiplicity

#
# Returns the "derivative" of a partition, denoted by $\partial\lambda$ in the
# paper.
#
def _partition_derivative(self):
  return Partition((part-1 for part in self))
Partition.derivative = _partition_derivative

#
# Returns the partition obtained from a given partition by removing a single
# part of size 1, throwing an exception if no such part exists.
#
def _partition_dot(self):
  assert self.length() >= 1 and self[-1] == 1
  return Partition((self[i] for i in range(len(self)-1)))
Partition.dot = _partition_dot

#
# Returns the partition obtained from a given partition by removing all parts of
# size 1, denoted by $\bar{\lambda}$ in the paper.
#
def _partition_reduction(self):
  return Partition((part for part in self if part >= 2))
Partition.reduction = _partition_reduction

In [5]:
################################################################################
# Chain Types.                                                                 #
################################################################################

#
# An implementation of "chain types" from the paper.
#
class ChainType:
  #
  # Creates a new chain type with a given rank and partition. In the paper, this
  # is denoted by $(\nu, \lambda)$
  #
  def __init__(self, rank, partition):
    assert rank >= 0
    # We allow `partition` to be specified as a partition or a list.
    partition = Partition(partition)
    self._rank = rank
    self._partition = partition

  def __eq__(self, other):
    return self._rank == other._rank and self._partition == other._partition
  def __hash__(self):
    return hash((self._rank, self._partition))
  def __repr__(self):
        return f"({self._rank}, {self._partition})"

  #
  # Returns the length of a chain type, denoted by `\len(\tau)` in the paper.
  #
  def length(self):
    return self._rank + self._partition.size()
  
  #
  # Returns the degree of a chain type, denoted by `\deg(\tau)` in the paper.
  #
  def degree(self):
    return self._rank + self._partition.size() - self._partition.length()

  #
  # Returns the rank of a chain type, denoted by `\rk(\tau)` in the paper.
  #
  def rank(self):
    return self._rank

  #
  # Returns the partition of a chain type.
  #
  def partition(self):
      return self._partition

  #
  # Returns the multiplicity of `j` in the partition of a chain type. In the
  # paper, this is denoted by $m_j(\tau)$.
  #
  def multiplicity(self, j):
    return self._partition.multiplicity(j)

  #
  # Returns the derivative of a chain type, denoted by $\partial\tau$ in the
  # paper.
  #
  def derivative(self):
    return ChainType(self._rank, self._partition.derivative())

  #
  # Returns the chain type obtained from this one by removing a single part of
  # size 1, assuming such a part exists. In the paper, this is denoted by
  # $\dot{\tau}$.
  #
  def dot(self):
    return ChainType(self._rank, self._partition.dot())

  #
  # Returns the reduction of a chain type, denoted by $\bar{\tau}$ in the paper.
  #
  def reduction(self):
    return ChainType(self._rank, self._partition.reduction())

#
# Returns the collection of all chain types with a given length, degree, and/or
# rank, corresponding to the sets $\calT[r]$, $\calT[r, d]$, and
# $\calT[r, d \mid \nu]$ from the paper.
#
def ChainTypes(r, d=None, nu=None):
  if d is None:
    return (
      ChainType(nu, mu)
    for nu in IntegerRange(r+1)
    for mu in Partitions(r-nu))
  elif nu is None:
    return (
      ChainType(nu, mu)
    for nu in IntegerRange(r+1)
    for mu in Partitions(r-nu, length=r-d))
  else:
    return (
      ChainType(nu, mu)
    for mu in Partitions(r-nu, length=r-d))

In [6]:
################################################################################
# Sequence arrangements.                                                       #
################################################################################

#
# An implementation of "sequence arrangements" from the paper. Here, we encode
# arrangements as pairs of increasing sublists of `[1, ..., r]` with the same
# size, rather than a set of ordered pairs. We refer to the first list as the
# "upper" list and the second as the "lower" list, referencing what they
# correspond to in the visual representation of the arrangement.
#
class SequenceArrangement:
  #
  # Creates a new sequence arrangement of a given length `r` from two strictly
  # increasing sublists of `[1, ..., r]` of the same size.
  #
  def __init__(self, length, lower, upper):
    assert length >= 0 and len(lower) == len(upper)
    # Check that `lower` and `upper` are both increasing sublists of
    # `[1, ..., n]`.
    assert is_increasing(lower) and is_increasing(upper)
    degree = len(lower)
    if degree >= 1:
      assert length >= 1
      assert (
        1 <= lower[0] and lower[-1] <= length and
        1 <= upper[0] and upper[-1] <= length
      )
    self._length = Integer(length)
    self._lower = lower
    self._upper = upper
    
  def __eq__(self, other):
    return (
      self._length == other._length and
      self._lower == other._lower and self._upper == other._upper
    )
  def __hash__(self):
    return hash((self._length, self._lower, self._upper))
  def __repr__(self):
    pairs = [(self._lower[i], self._upper[i]) for i in range(len(self._lower))]
    return f"({self._length}, {pairs})"

  #
  # Returns the length of a sequence arrangement, denoted by `\len(\alpha)` in
  # the paper.
  #
  def length(self):
    return self._length

  #
  # Returns the degree of a sequence arrangement, denoted by `\deg(\alpha)` in
  # the paper.
  #
  def degree(self):
    return Integer(len(self._lower))

  #
  # Returns the `i`th element of the lower/upper list. In addition, if `i` is
  # zero, returns zero, and if `i` is equal to the degree of the arrangement
  # plus one, returns the length of the arrangement plus one.
  #
  def lower(self, i):
    if i == 0:
      return 0
    elif i == self.degree()+1:
      return self.length()+1
    else:
      return self._lower[i-1]
  def upper(self, i):
    if i == 0:
      return 0
    elif i == self.degree()+1:
      return self.length()+1
    else:
      return self._upper[i-1]

  #
  # Returns the subrange of the set `[0, ..., r]` lying in between two
  # successive entries in the lower/upper list, where `r` is the length of this
  # arrangement.
  #
  def LowerRange(self, i):
    return IntegerRange(self.lower(i), self.lower(i+1))
  def UpperRange(self, i):
    return IntegerRange(self.upper(i), self.upper(i+1))

  #
  # Returns the number of elements of  `[1, ..., r]` lying strictly in between
  # two successive entries in the lower/upper list, where `r` is the length of
  # this arrangement.
  #
  def lower_delta(self, i):
    return self.lower(i+1) - self.lower(i) - 1
  def upper_delta(self, i):
    return self.upper(i+1) - self.upper(i) - 1

  #
  # Returns the set of pairs that would be used in the paper to represent this
  # arrangement.
  #
  def Pairs(self):
    return set((self._lower[i], self._upper[i]) for i in range(self.degree()))

  #
  # Returns the derivative of an arrangement, denoted by $\partial\alpha$ in the
  # paper.
  #
  def derivative(self):
    length = self.degree()
    lower = [i+1 for i in IntegerRange(length) if self._lower[i] in self._upper]
    upper = [i+1 for i in IntegerRange(length) if self._upper[i] in self._lower]
    return SequenceArrangement(length, lower, upper)

  #
  # Returns true if the given permutation contains this arrangement,
  # corresponding to the notation $\alpha \mid \pi$ in the paper.
  #
  def divides(self, pi):
    return all(
      pi(self._lower[i]) == self._upper[i] for i in range(self.degree())
    )

  #
  # Returns the chain type of this arrangement, denoted by $\typ(\alpha)$ in the
  # paper.
  #
  def chain_type(self):
    # We exploit the fact that the chain type may be computed from the lengths
    # of successive derivatives. More precisely, if $\typ(\alpha) = \tau$, then
    # $m_j(\tau) = \len(\partial^{j-1}\tau) - 2\len(\partial^j\tau) +
    # \len(\partial^{j+1}\tau)$ for all $j \ge 1$ and $\rk(\tau) =
    # \lim_{j \to \infty} \len(\partial^j\tau)$.
    alpha = self
    lengths = []
    while True:
      lengths.append(alpha.length())
      if alpha.degree() == alpha.length():
        break
      alpha = alpha.derivative()
    rank = lengths[-1]; lengths.append(rank)
    parts = []
    for j in reversed(range(len(lengths)-2)):
      parts.extend((lengths[j]-2*lengths[j+1]+lengths[j+2]) * [j+1])
    return ChainType(rank, Partition(parts))

#
# Returns the set of sequence arrangements of a given length and/or degree,
# denoted by $\calA[r]$ or $\calA[r, d]$ in the paper.
#
def SequenceArrangements(length, degree=None):
  if degree is None:
    return (
      SequenceArrangement(length, lower, upper)
    for degree in IntegerRange(length+1)
    for lower in Sublists(length, degree) for upper in Sublists(length, degree))
  else:
    return (
      SequenceArrangement(length, lower, upper)
    for lower in Sublists(length, degree) for upper in Sublists(length, degree))

#
# Returns an arbitrary representative of a given chain type.
#
def _chain_type_representative(self):
  rank = self.rank(); partition = self.partition()
  lower = [i+1 for i in IntegerRange(rank)]
  upper = [i+1 for i in IntegerRange(rank)]
  offset = rank
  for part in partition:
    lower.extend([offset+i+1 for i in IntegerRange(part-1)])
    upper.extend([offset+i+2 for i in IntegerRange(part-1)])
    offset += part
  return SequenceArrangement(offset, lower, upper)
ChainType.representative = _chain_type_representative

#
# Returns the set of sequence arrangements with a given chain type, denoted by
# $\calA(\tau)$ in the paper.
#
def _chain_type_SequenceArrangements(self):
  return (
    alpha for alpha in SequenceArrangements(self.length(), self.degree())
  if alpha.chain_type() == self)
ChainType.SequenceArrangements = _chain_type_SequenceArrangements

In [7]:
################################################################################
# Direct computation of $h^\tau(n, k)$ and $\kappa^\tau(n, k)$.                #
################################################################################

#
# Computes the value $h^\tau(n, k)$ directly from the definition.
#
def cp_h(tau, n, k):
  r = tau.length(); d = tau.degree()
  return Integer(sum(
    prod(
      binomial(sum(ns[l] for l in alpha.LowerRange(i)), ks[i]) *
      binomial(sum(ns[l] for l in alpha.UpperRange(i)), ks[i])
    for i in range(d+1))
  for alpha in tau.SequenceArrangements()
  for ns in IntegerVectors(n, r+1)
  for ks in IntegerVectors(k, d+1)))

#
# Computes the value $\kappa^\tau(n, k)$ directly from the definition.
#
def cp_kappa(tau, n, k):
  r = tau.length(); d = tau.degree()
  return Integer(sum(
    prod(
      binomial(
        ns[i] + alpha.lower_delta(i),
        sum(ks[l] for l in alpha.LowerRange(i)) + alpha.lower_delta(i)
      ) * binomial(
        ns[i] + alpha.upper_delta(i),
        sum(ks[l] for l in alpha.UpperRange(i)) + alpha.upper_delta(i)
      )
    for i in range(d+1))
  for alpha in tau.SequenceArrangements()
  for ns in IntegerVectors(n, d+1)
  for ks in IntegerVectors(k, r+1)))

In [8]:
################################################################################
# Computation of $a^\lambda(n, k)$ from $h^\tau(n, k)$.                        #
################################################################################

#
# Computes the value $\vartheta_\tau^\lambda$ from the paper.
#
def cp_theta(tau, lam):
  lam = Partition(lam)
  r = tau.length(); l = lam.size();
  lam_deriv = lam.derivative(); alpha = tau.representative()
  result = 0
  for mu in Partitions(r, inner=lam_deriv, outer=lam):
    chi = cp_chi(mu)
    result += sum(
      chi(sigma) for sigma in SymmetricGroup(r) if alpha.divides(sigma)
    )
  return Integer((-1)^(l-r) * result)

#
# Computes the value $a^\lambda(n, k)$ using the formula in terms of
# $h^\tau(n, k)$.
#
def cp_a_from_h(lam, n, k, *, h_fun=cp_h):
  lam = Partition(lam)
  l = lam.size(); l_prime = lam.derivative().size()
  result = 0
  for r in IntegerRange(l_prime, l+1):
    for d in IntegerRange(max(0, r-(n-k)), r+1):
      result += factorial(n-k-r+d) * sum(
        cp_theta(tau, lam) * h_fun(tau, n-r, k-d)
      for tau in ChainTypes(r, d)) / factorial(n)
  return result

In [9]:
################################################################################
# Computation of the polynomials $P^\tau(z, w)$.                               #
################################################################################

#
# The ring of formal power series in $x$ and $y$, together with several power
# series defined in the paper.
#
XYSeries.<var_x, var_y> = LazyPowerSeriesRing(QQ, sparse=False)
cp_Q = XYSeries(sqrt((1-var_x-var_y)^2-4*var_x*var_y))
cp_u = XYSeries((1+var_x-var_y+cp_Q)/(2*cp_Q))
cp_v = XYSeries((1-var_x+var_y-cp_Q)/(2*cp_Q))
cp_xi = XYSeries(var_x/cp_Q^2)
cp_eta = XYSeries(var_y/cp_Q^2)

#
# The polynomial ring used for the polynomials $P^\tau(z, w)$. We also define
# a ring of polynomials in variables $Z$, $W$, and $T$, which will be necessary
# for circumventing the fact that the fraction fields of lazy power series are
# not yet implemented.
#
ZWPolynomial.<var_z, var_w> = PolynomialRing(ZZ)
_ZWTPolynomial.<var_Z_, var_W_, var_T_> = PolynomialRing(ZZ)

#
# Computes the value $\zeta_{\omega'}^\omega$ defined in the paper.
#
def cp_zeta(omega_prime, omega):
  assert omega_prime.length() == omega.length()
  beta_prime = omega_prime.representative()
  return Integer(sum(
    1 for beta in omega.SequenceArrangements()
  if beta.Pairs() <= beta_prime.Pairs()))

#
# Computes the polynomial $P^\tau(z, w)$ from the paper. Previously computed
# polynomials are cached for later use.
#
def cp_P(tau):
  global _cp_P_cache
  # If the requested polynomial has already been computed, return it.
  P = _cp_P_cache.get(tau)
  if P is not None:
    return P
  # Otherwise, compute the polynomial using the appropriate case in the
  # recursive definition, and save the result to the cache.
  r = tau.length(); d = tau.degree(); nu = tau.rank(); m = tau.multiplicity(1)
  omega = tau.derivative(); s = omega.degree()
  if r == d:
    P = ZWPolynomial(1)
  elif m >= 1:
    ell = r-d-1; gamma = nu+1
    P_dot = cp_P(tau.dot())
    P = ZWPolynomial((
      (
        ell*var_z*var_w + (3*ell+gamma)*var_z +
        (r+2*ell)*var_w + (4*ell+2*gamma)
      ) * P_dot +
      -var_z*(var_z+1)*(var_w+2) * derivative(P_dot, var_z) +
      -var_w*(var_w+1)*(var_z+2) * derivative(P_dot, var_w)
    ) / m)
  else:
    P = ZWPolynomial(sum(
      (-var_z)^(var_prime-s) * var_w^((var_prime-nu_prime)-(s-nu)) *
      (var_w+1)^(nu_prime-nu) * sum(
        cp_zeta(omega_prime, omega) *
        cp_P(omega_prime)(var_z*var_w + var_z + var_w, var_w)
      for omega_prime in ChainTypes(d, var_prime, nu_prime))
    for var_prime in IntegerRange(s, d+1)
    for nu_prime in IntegerRange(nu, nu+var_prime-s+1)))
  _cp_P_cache[tau] = P
  return P
_cp_P_cache = {}

#
# Computes the power series $H^\tau$ via the formula in terms of $P^\tau$.
#
def cp_H(tau):
  r = tau.length(); d = tau.degree(); nu = tau.rank()
  # Because the fraction field of `XYSeries` is not implemented, we must first
  # replace `P` with a homogeneous polynomial.
  P = cp_P(tau)
  P_homogenized = _ZWTPolynomial(var_T_^(r-d) * P(var_Z_/var_T_, var_W_/var_T_))
  return XYSeries(
    cp_u^(r-d) * (cp_v+1)^(nu-r) * cp_Q^(-1-d) *
    P_homogenized(-cp_eta/cp_u, cp_v+1, cp_eta)
  )

#
# Computes the power series $K^\tau$ via the formula in terms of $P^\tau$.
#
def cp_K(tau):
  r = tau.length(); d = tau.degree(); nu = tau.rank()
  P = cp_P(tau)
  P_homogenized = _ZWTPolynomial(var_T_^(r-d) * P(var_Z_/var_T_, var_W_/var_T_))
  return XYSeries(
    cp_u^(r-d) * (cp_v+1)^(d+nu-2*r) * cp_Q^(-1-r) *
    P_homogenized((cp_v+1)*(cp_u-cp_v-1)/cp_u, cp_v+1, cp_eta)
  )

In [10]:
################################################################################
# Implementation of the lemma for computing series coefficients of polynomials #
# in $u$ and $v$ times powers of $Q$.                                          #
################################################################################

#
# Polynomials in $u$ and $v$. Here, the variable `var_v1_` represents $v+1$.
#
UVPolynomial.<var_u, var_v> = PolynomialRing(ZZ)
_XiEtaUVPolynomial.<var_xi_, var_eta_, var_u_, var_v1_> = PolynomialRing(ZZ)
_ReducedUVPolynomial = _XiEtaUVPolynomial.quotient((
  var_u_^2-var_u_-var_xi_, var_v1_^2-var_v1_-var_eta_
))

#
# Polynomials in $j$ and $k$, used in formulas for the series coefficients of
# polynomials in $u$ and $v$ times powers of $Q$.
#
_JKPolynomial.<var_j_, var_k_> = PolynomialRing(ZZ)

#
# Implementations of the functions $F_s^\uparrow(r, j)$ and
# $F_s^\downarrow(r, j)$ defined in the paper.
#
def cp_F_1_down(r, j):
  if j >= 0:
    return prod(r-i for i in IntegerRange(0, j))
  else:
    return 1/prod(r-i for i in IntegerRange(j, 0))
def cp_F_1_up(r, j):
  if j >= 0:
    return prod(r+i for i in IntegerRange(1, j+1))
  else:
    return 1/prod(r+i for i in IntegerRange(j+1, 1))
def cp_F_2_down(r, j):
  if j >= 0:
    return prod(r-2*i for i in IntegerRange(0, j))
  else:
    return 1/prod(r-2*i for i in IntegerRange(j, 0))
def cp_F_2_up(r, j):
  if j >= 0:
    return prod(r+2*i for i in IntegerRange(1, j+1))
  else:
    return 1/prod(r+2*i for i in IntegerRange(j+1, 1))

#
# Encodes pairs of polynomials in $j$ and $k$ used for computing series
# coefficients of polynomials in $u$ and $v$ times powers of $Q$.
#
class SeriesPolynomials:
  #
  # Given a polynomial $F \in \Z[u, v]$ of $u$-degree at most $a$ and $v$-degree
  # at most $b$, computes series polynomials for the generating function
  # $FQ^{-1-d}$.
  #
  def __init__(self, F, d, a, b):
    F = UVPolynomial(F)
    # Make sure $F$ satisfies the required degree constraints.
    assert F.degree(var_u) <= a and F.degree(var_v) <= b
    # Express $F$ as a polynomial in $u$, $v+1$, $\xi$, and $\eta$ with
    # $u$- and $(v+1)$-degree at most $1$.
    F_bar = _ReducedUVPolynomial(F(var_u_, var_v1_-1)).lift()
    # We apply a slightly more explicit version of the theorem from the paper.
    a_i = [a_0, a_1] = [floor((a-i)/2) for i in IntegerRange(2)]
    b_i = [b_0, b_1] = [floor((b-i)/2) for i in IntegerRange(2)]
    coeffs = [
      [
        [[ # (eps, delta) = (0, 0)
          2*F_bar.monomial_coefficient(var_xi_^alpha*var_eta_^beta) +
          F_bar.monomial_coefficient(var_u_*var_xi_^alpha*var_eta_^beta) +
          F_bar.monomial_coefficient(var_v1_*var_xi_^alpha*var_eta_^beta)
        for beta in IntegerRange(b_0+1)]
        for alpha in IntegerRange(a_0+1)],
        [[ # (eps, delta) = (0, 1)
          2*F_bar.monomial_coefficient(var_v1_*var_xi_^alpha*var_eta_^beta) +
          F_bar.monomial_coefficient(var_u_*var_v1_*var_xi_^alpha*var_eta_^beta)
        for beta in IntegerRange(b_1+1)]
        for alpha in IntegerRange(a_0+1)],
      ], [
        [[ # (eps, delta) = (1, 0)
          2*F_bar.monomial_coefficient(var_u_*var_xi_^alpha*var_eta_^beta) +
          F_bar.monomial_coefficient(var_u_*var_v1_*var_xi_^alpha*var_eta_^beta)
        for beta in IntegerRange(b_0+1)]
        for alpha in IntegerRange(a_1+1)],
        [[ # (eps, delta) = (1, 1)
          F_bar.monomial_coefficient(var_u_*var_v1_*var_xi_^alpha*var_eta_^beta)
        for beta in IntegerRange(b_1+1)]
        for alpha in IntegerRange(a_1+1)],
      ]
    ]
    (D_0, D_1) = (floor((d+1-i)/2) for i in IntegerRange(2))
    (A_0, A_1) = (floor((d+a-i)/2) for i in IntegerRange(2))
    (B_0, B_1) = (floor((d+b-i)/2) for i in IntegerRange(2))
    (C_0, C_1) = (floor((d+a+b-i)/2) for i in IntegerRange(2))
    g_hat_0 = _JKPolynomial(sum(
      coeffs[eps][delta][alpha][beta] *
      2^(C_0-D_0-alpha-beta) *
      cp_F_2_down(2*C_0-1, C_0-D_0-alpha-beta-eps*delta) *
      cp_F_1_up(var_j_+var_k_+d, alpha+beta+eps*delta) *
      cp_F_1_down(var_j_, alpha) *
      cp_F_1_down(var_k_, beta) *
      cp_F_1_down(var_j_+B_0, B_0-D_0-beta+eps*(1-delta)) *
      cp_F_1_down(var_k_+A_0, A_0-D_0-alpha+delta*(1-eps))
    for (eps, delta) in ([(0, 0), (1, 1)] if d % 2 == 0 else [(1, 0), (0, 1)])
    for alpha in IntegerRange(a_i[eps]+1)
    for beta in IntegerRange(b_i[delta]+1)))
    g_hat_1 = _JKPolynomial(sum(
      coeffs[eps][delta][alpha][beta] *
      2^(C_1-D_1-alpha-beta-eps-delta+eps*delta) *
      cp_F_1_down(C_1, C_1-D_1-alpha-beta-eps*delta) *
      cp_F_1_up(var_j_+var_k_+d, alpha+beta+eps*delta) *
      cp_F_1_down(var_j_, alpha) *
      cp_F_1_down(var_k_, beta) *
      cp_F_2_down(2*(var_j_+B_1)+1, B_1-D_1-beta+eps*(1-delta)) *
      cp_F_2_down(2*(var_k_+A_1)+1, A_1-D_1-alpha+delta*(1-eps))
    for (eps, delta) in ([(1, 0), (0, 1)] if d % 2 == 0 else [(0, 0), (1, 1)])
    for alpha in IntegerRange(a_i[eps]+1)
    for beta in IntegerRange(b_i[delta]+1)))
    self._F = F; self._d = d
    self._a = a; self._b = b
    self._g_hat_0 = g_hat_0; self._g_hat_1 = g_hat_1

  def __repr__(self):
    F = self._F(var('u'), var('v')); d = self._d
    return f"Series polynomials for ({F})*Q^({-1-d})"

  #
  # Getters for the polynomials $\hat{g}_i(j, k)$ used for computing series
  # coefficients.
  #
  def g_hat_0(self, j, k):
    return self._g_hat_0(j, k)
  def g_hat_1(self, j, k):
    return self._g_hat_1(j, k)

  #
  # Returns the power series $G = F Q^{-1-d}$ described by this
  # `SeriesPolynomials` instance.
  #
  def G(self):
    F = self._F; d = self._d
    return XYSeries(F(cp_u, cp_v) * cp_Q^(-1-d))

  #
  # Computes the coefficient $g(j, k)$ of $x^jy^k$ in the series described by
  # this `SeriesPolynomials` instance using the polynomials $\hat{g}_i(j, k)$.
  # We require that $j+k \ge -d$.
  #
  def g(self, j, k):
    d = self._d; a = self._a; b = self._b
    assert j+k >= -d
    if j < 0 or k < 0:
      return 0
    (D_0, D_1) = (floor((d+1-i)/2) for i in IntegerRange(2))
    (A_0, A_1) = (floor((d+a-i)/2) for i in IntegerRange(2))
    (B_0, B_1) = (floor((d+b-i)/2) for i in IntegerRange(2))
    (C_0, C_1) = (floor((d+a+b-i)/2) for i in IntegerRange(2))
    return (
        (self._g_hat_0(j, k) * 2^(-C_0-1) * factorial(j+k+D_0) / (
            bifactorial(2*C_0-1) *
            factorial(j+B_0) * factorial(k+A_0)
        ) if j+B_0 >= 0 and k+A_0 >= 0 else 0) +
        (self._g_hat_1(j, k) * 2^(-C_1-1) * bifactorial(2*(j+k+D_1)+1) / (
            factorial(C_1) *
            bifactorial(2*(j+B_1)+1) * bifactorial(2*(k+A_1)+1)
        ) if C_1 >= 0 else 0)
    ) * factorial(j+k+d) / (factorial(j)*factorial(k))

In [11]:
################################################################################
# Computation of the polynomials $\hat{h}_i^\tau(n, k)$ and                    #
# $\hat{a}_i^{l,\tau}(n, k)$.                                                  #
################################################################################

#
# Polynomials in $n$ and/or $k$. We use capital letters for the univariate
# polynomials.
#
NKPolynomial.<var_n, var_k> = PolynomialRing(ZZ)
NPolynomial.<var_N> = PolynomialRing(ZZ)
KPolynomial.<var_K> = PolynomialRing(ZZ)
NPolynomialQQ.<var_N_QQ> = PolynomialRing(QQ)
KPolynomialQQ.<var_K_QQ> = PolynomialRing(QQ)

#
# Encodes pairs of polynomials $\hat{h}_i^\tau(n, k)$ used for computing the
# values $h^\tau(n, k)$.
#
class HPolynomials:
  #
  # Given a reduced chain type $\tau$ with $\rk(\tau) < \deg(\tau)$, computes
  # the polynomials $\hat{h}_i(n, k)$ used in the formula for the series
  # coefficients of $H^\tau(n, k)$.
  #
  def __init__(self, tau):
    r = tau.length(); d = tau.degree(); nu = tau.rank()
    assert tau.multiplicity(1) == 0 and nu < d
    P_hat = ZWPolynomial(cp_P(tau) / (2*(var_w+1)))
    series_polys = SeriesPolynomials(
      var_u^(r-d)*var_v^(r-nu-2)*P_hat(-1/var_u, 1/var_v), 2*(nu+1)-d,
      r-d, r-nu-2
    )
    self._tau = tau
    self._h_hat_0 = NKPolynomial(
      series_polys._g_hat_0(var_n-var_k, var_k+d-nu-1)
    )
    self._h_hat_1 = NKPolynomial(
      series_polys._g_hat_1(var_n-var_k, var_k+d-nu-1)
    )

  def __repr__(self):
    tau = self._tau
    return f"H-polynomials for {tau}"

  #
  # Getters for the polynomials $\hat{h}_i^\tau(j, k)$.
  #
  def h_hat_0(self, n, k):
    return self._h_hat_0(n, k)
  def h_hat_1(self, n, k):
    return self._h_hat_1(n, k)

  #
  # Returns the power series $H^\tau(n, k)$ described by this `HPolynomials`
  # instance.
  #
  def H(self):
    return cp_H(self._tau)

  #
  # Computes the value $h^\tau(n, k)$ using the formula in terms of the
  # polynomials $\hat{h}_i^\tau(n, k)$. We require that $n \ge -\nu-1$.
  #
  def h(self, n, k):
    tau = self._tau
    r = tau.length(); d = tau.degree(); nu = tau.rank()
    assert n >= -nu-1
    if n < k or k < nu+1-d:
      return 0
    (r_0, r_1) = (floor((r-i)/2) for i in IntegerRange(2))
    (d_0, d_1) = (floor((d+1-i)/2) for i in IntegerRange(2))
    (nu_0, nu_1) = (floor((nu-i)/2) for i in IntegerRange(2))
    (b_0, b_1) = (floor((nu-d+r-i)/2) for i in IntegerRange(2))
    return (
      self._h_hat_0(n, k) * 2^(-r+d-nu_0) * factorial(n+d_0) / (
        bifactorial(2*(r-d+nu_0)-1) *
        factorial(k+r_0) * factorial(n-k+b_0)
      ) +
      self._h_hat_1(n, k) * 2^(-r+d-nu_1) * bifactorial(2*(n+d_1)+1) / (
        factorial(r-d+nu_1) *
        bifactorial(2*(k+r_1)+1) * bifactorial(2*(n-k+b_1)+1)
      )
    ) * factorial(n+nu+1) / (factorial(k+d-nu-1)*factorial(n-k))

  #
  # Computes the polynomials $\hat{a}_i^{\tau',l})(n, k)$, where $\tau'$ is the
  # unique chain type such that $\bar{\tau}' = \tau$ and $m_1(\tau') = m$.
  #
  def a_hat_0(self, m, l, n, k):
    l_0 = floor(l/2)
    tau = self._tau
    r_bar = tau.length(); r = r_bar+m; d = tau.degree(); nu = tau.rank()
    assert l >= r
    r_bar_0 = floor(r_bar/2); d_0 = floor((d+1)/2); nu_0 = floor(nu/2)
    b_bar_0 = floor((nu-d+r_bar)/2)
    return (
      self.h_hat_0(n-r, k-d) *
      2^(l_0-r_bar+d-nu_0) * cp_F_2_down(2*l_0-1, l_0-r_bar+d-nu_0) *
      cp_F_1_down(n, m) *
      cp_F_1_up(n-l+1, l-r+nu) * cp_F_1_up(n-l, l+d_0-r) *
      cp_F_1_down(k-1, nu) * cp_F_1_down(k, d-r_bar_0) *
      cp_F_1_down(n-k+l_0, l_0+r-d-b_bar_0)
    )
  def a_hat_1(self, m, l, n, k):
    l_1 = floor((l-1)/2)
    tau = self._tau
    r_bar = tau.length(); r = r_bar+m; d = tau.degree(); nu = tau.rank()
    assert l >= r
    r_bar_1 = floor((r_bar-1)/2); d_1 = floor(d/2); nu_1 = floor((nu-1)/2)
    b_bar_1 = floor((nu-d+r_bar-1)/2)
    return (
      self.h_hat_1(n-r, k-d) *
      2^(l_1-r_bar+d-nu_1) * cp_F_1_down(l_1, l_1-r_bar+d-nu_1) *
      cp_F_1_down(n, m) *
      cp_F_1_up(n-l+1, l-r+nu) * cp_F_2_up(2*(n-l)+1, l+d_1-r) *
      cp_F_1_down(k-1, nu) * cp_F_2_down(2*k-1, d-r_bar_1-1) *
      cp_F_2_down(2*(n-k+l_1)+1, l_1+r-d-b_bar_1)
    )

#
# Computes an `HPolynomials` instance for the given chain type, caching
# previously computed instances.
#
def h_polynomials(tau):
  global _h_polynomials_cache
  h_polys = _h_polynomials_cache.get(tau)
  if h_polys is not None:
    return h_polys
  h_polys = HPolynomials(tau)
  _h_polynomials_cache[tau] = h_polys
  return h_polys
_h_polynomials_cache = {}

In [12]:
################################################################################
# Computation of the polynomials $\hat{a}_i^\lambda(n, k)$ and                 #
# $B_j^\lambda(k)$.                                                            #
################################################################################

#
# Encodes pairs of polynomials $\hat{a}_i^\lambda(n, k)$ together with lists of
# polynomials $B_j^\lambda(k)$ used for computing the values $a^\lambda(n, k)$.
#
class APolynomials:
  #
  # Computes $\hat{a}_i^\lambda(n, k)$ and $B_j^\lambda(k)$ for a given
  # partition $\lambda$.
  #
  def __init__(self, lam):
    lam = Partition(lam); lam_prime = lam.derivative()
    l = lam.size(); l_prime = lam_prime.size(); lam_1 = lam.get_part(0)
    assert l >= 1
    (l_0, l_1) = (floor((l-i)/2) for i in IntegerRange(2))
    a_hat_0 = NKPolynomial(0)
    a_hat_1 = NKPolynomial(0)
    # We will compute the polynomials $B_j^\lambda(k)$ from the two lists below.
    a_tilde_0 = [KPolynomial(0) for j in IntegerRange(l-1)]
    a_tilde_1 = [KPolynomial(0) for j in IntegerRange(l-1)]
    # Add contributions from chain types of positive degree.
    for r in IntegerRange(l_prime, l+1):
      for d in IntegerRange(1, r+1):
        for nu in IntegerRange(d+1):
          for tau in ChainTypes(r, d, nu):
            m = tau.multiplicity(1)
            coeff = Integer(cp_theta(tau, lam) / factorial(m))
            if nu < d:
              h_polys = h_polynomials(tau.reduction())
              poly_0 = NKPolynomial(coeff * h_polys.a_hat_0(m, l, var_n, var_k))
              poly_1 = NKPolynomial(coeff * h_polys.a_hat_1(m, l, var_n, var_k))
            elif d % 2 == 0:
              d_0 = Integer(d/2)
              poly_0 = NKPolynomial(
                coeff * 2^(l_0-d_0) * cp_F_2_down(2*l_0-1, l_0-d_0) *
                cp_F_1_down(var_n, l-1) * cp_F_1_up(var_n-l, l-r+d_0) *
                cp_F_1_down(var_k-1, d-1) * cp_F_1_down(var_k, d-d_0) *
                cp_F_1_down(var_n-var_k+l_0, l_0-d_0+r-d)
              )
              poly_1 = NKPolynomial(0)
            else:
              d_1 = Integer((d-1)/2)
              poly_0 = NKPolynomial(0)
              poly_1 = NKPolynomial(
                coeff * 2^(l_1-d_1) * cp_F_1_down(l_1, l_1-d_1) *
                cp_F_1_down(var_n, l-1) * cp_F_2_up(2*(var_n-l)+1, l-r+d_1) *
                cp_F_1_down(var_k-1, d-1) * cp_F_2_down(2*var_k-1, d-d_1-1) *
                cp_F_2_down(2*(var_n-var_k+l_1)+1, l_1-d_1+r-d)
              )
            a_hat_0 += poly_0; a_hat_1 += poly_1
            for j in IntegerRange(r-d, l-1):
              a_tilde_0[j] += KPolynomial(poly_0(j+var_K, var_K))
              a_tilde_1[j] += KPolynomial(poly_1(j+var_K, var_K))
    # If $\partial\lambda$ has a single row, we need to add one more term,
    # corresponding to the chain types of degree 0.
    if lam_prime.length() <= 1:
      poly_0 = NKPolynomial(
        -(-1)^(l-lam_1) * 2^l_0 * bifactorial(2*l_0-1) *
        cp_F_1_down(var_n, l-1) * cp_F_1_up(var_n-l, l-lam_1) *
        cp_F_1_down(var_n-var_k+l_0, l_0+lam_1-1)
      )
      a_hat_0 += poly_0
      for j in IntegerRange(lam_1-1, l-1):
        a_tilde_0[j] += KPolynomial(poly_0(j+var_K, var_K))
    self._lam = lam
    self._a_hat_0 = a_hat_0
    self._a_hat_1 = a_hat_1
    self._B = [
      KPolynomialQQ((
        a_tilde_0[j](var_K_QQ) * (
          2^(-l_0) * cp_F_1_up(var_K_QQ, j-l)
        ) / (bifactorial(2*l_0-1) * factorial(j+l_0)) +
        a_tilde_1[j](var_K_QQ) * (
          2^(-l_1) * cp_F_2_up(2*var_K_QQ-1, j-l+1)
        ) / (factorial(l_1) * bifactorial(2*(j+l_1)+1))
      ) * cp_F_1_up(var_K_QQ-1, j-l+2))
    for j in IntegerRange(l-1)]

  def __repr__(self):
    lam = self._lam
    return f"a-polynomials for {lam}"

  #
  # Getters for the polynomials $\hat{a}_i^\lambda(n, k)$.
  #
  def a_hat_0(self, n, k):
    return self._a_hat_0(n, k)
  def a_hat_1(self, n, k):
    return self._a_hat_1(n, k)

  #
  # Getters for the leading coefficients of the polynomials
  # $\tilde{a}_i^{\lambda,j}(k)$.
  #
  def leading_coefficient_0(self):
    l = self._lam.size(); l_0 = floor(l/2)
    return self._a_hat_0.monomial_coefficient((var_n*var_k)^(l+l_0-1))
  def leading_coefficient_1(self):
    l = self._lam.size(); l_1 = floor((l-1)/2)
    return self._a_hat_1.monomial_coefficient((var_n*var_k)^(l+l_1-1))

  #
  # Computes the value $a^\lambda(n, k)$ using the formula in terms of the
  # polynomials $\hat{a}_i^\lambda(n, k)$ and $\tilde{a}_i^{\lambda,j}(k)$.
  #
  def a(self, n, k):
    lam = self._lam; lam_prime = lam.derivative()
    l = lam.size(); l_prime = lam_prime.size(); lam_1 = lam.get_part(0)
    (l_0, l_1) = (floor((l-i)/2) for i in IntegerRange(2))
    assert n >= l+lam_1 and k >= 0
    if k == 0 or n < k:
      return 0
    if n-k < l-1:
      return self._B[n-k](k) / factorial(n)
    return (
      self._a_hat_0(n, k) * 2^(-l_0) * factorial(n-l) / (
        bifactorial(2*l_0-1) *
        factorial(k) * factorial(n-k+l_0)
      ) +
      self._a_hat_1(n, k) * 2^(-l_1) * bifactorial(2*(n-l)+1) / (
        factorial(l_1) *
        bifactorial(2*k-1) * bifactorial(2*(n-k+l_1)+1)
      )
    ) * factorial(n-l+1) / (factorial(k-1) * factorial(n))

  #
  # Computes the polynomial $A_k^\lambda(n)$ using the formula in terms of the
  # polynomials $\hat{a}_i^\lambda(n, k)$ and $\tilde{a}_i^{\lambda,j}(k)$.
  #
  def A(self, k):
    lam = self._lam; l = lam.size()
    (l_0, l_1) = (floor((l-i)/2) for i in IntegerRange(2))
    assert k >= 0
    if k == 0:
      return NPolynomialQQ(0)
    return NPolynomialQQ((
      self.a_hat_0(var_N_QQ, k) * (
        2^(-l_0) * cp_F_1_down(var_N_QQ-l, k-l-l_0)
      ) / (bifactorial(2*l_0-1) * factorial(k)) +
      self.a_hat_1(var_N_QQ, k) * (
        2^(-l_1) * cp_F_2_down(2*(var_N_QQ-l)+1, k-l-l_1)
      ) / (factorial(l_1) * bifactorial(2*k-1))
    ) / (factorial(k-1)*cp_F_1_down(var_N_QQ, l-1)))

  #
  # Computes the polynomial $B_j^\lambda(k)$ using the internal table when
  # $j < l-1$ and the formula in terms of the polynomials
  # $\hat{a}_i^\lambda(n, k)$ and $\tilde{a}_i^{\lambda,j}(k)$ otherwise.
  #
  def B(self, j):
    lam = self._lam; l = lam.size()
    (l_0, l_1) = (floor((l-i)/2) for i in IntegerRange(2))
    assert j >= 0
    if j < l-1:
      return self._B[j]
    return KPolynomialQQ((
      self.a_hat_0(j+var_K_QQ, var_K_QQ) * (
        2^(-l_0) * cp_F_1_up(var_K_QQ, j-l)
      ) / (bifactorial(2*l_0-1) * factorial(j+l_0)) +
      self.a_hat_1(j+var_K_QQ, var_K_QQ) * (
        2^(-l_1) * cp_F_2_up(2*var_K_QQ-1, j-l+1)
      ) / (factorial(l_1) * bifactorial(2*(j+l_1)+1))
    ) * cp_F_1_up(var_K_QQ-1, j-l+2))
  
  #
  # Returns either a pair $(n, k)$ such that $a^\lambda(n, k) < 0$ or `None`
  # if no such pair exists.
  #
  def positivity_counterexample(self):
    lam = self._lam; l = lam.size(); lam_1 = lam.get_part(0)
    (l_0, l_1) = (floor((l-i)/2) for i in IntegerRange(2))
    lead_coeff_0 = self.leading_coefficient_0()
    lead_coeff_1 = self.leading_coefficient_1()
    assert (-1)^l * lead_coeff_0 > 0 and (-1)^(l+1) * lead_coeff_1 > 0
    # We will choose an integer $t$ sufficiently large with the property that
    # $a^\lambda(j+k+l-1+t, k+t) \ge 0$ for all $j, k \ge 0$. We start by
    # choosing $t$ to ensure that all the coefficients of
    # $\hat{a}_i^\lambda(j+k+l-1+t, k+t)$ have the same sign.
    a_hat_0_abs = sign(lead_coeff_0)*self._a_hat_0
    a_hat_1_abs = sign(lead_coeff_1)*self._a_hat_1
    t = 1
    while not (
      has_nonnegative_coefficients(_JKPolynomial(a_hat_0_abs(
        var_j_+var_k_+l-1+t, var_k_+t
      ))) and
      has_nonnegative_coefficients(_JKPolynomial(a_hat_1_abs(
        var_j_+var_k_+l-1+t, var_k_+t
      )))
    ):
      t += 1
    # Now we need to further increase `t` to ensure that $a^\lambda(n, k)$
    # itself is nonnegative.
    if l % 2 == 0:
      diff_poly = NKPolynomial(
        2^(2*l-2) * a_hat_0_abs^2 -
        l_0 * (var_n-l+1) * (2*(var_n-var_k)+l+1) * (2*var_k+1) * a_hat_1_abs^2
      )
      while not has_nonnegative_coefficients(
        _JKPolynomial(diff_poly(var_j_+var_k_+l-1+t, var_k_+t))
      ):
        t += 1
    else:
      diff_poly = NKPolynomial(
        var_k*(2*(var_n-l)+1) * a_hat_1_abs^2 -
        2^(2*l) * l * (var_n-var_k+l_1+1) * a_hat_0_abs^2
      )
      while not has_nonnegative_coefficients(
        _JKPolynomial(diff_poly(var_j_+var_k_+l-1+t, var_k_+t))
      ):
        t += 1
    # At this point, we must have $a^\lambda(j+k, k) \ge 0$ for all $j \ge l-1$
    # and $k \ge t$. Thus, we need only check that the polynomials
    # $A_k^\lambda(n)$ for $k < t$ and $B_j^\lambda(k)$ for $j < l-1$ attain
    # only nonnegative values.
    for k in IntegerRange(t):
      A = self.A(k)
      if A == 0:
        continue
      elif A.leading_coefficient() < 0:
        # If `A` has a negative leading coefficient, it eventually achieves a
        # negative value.
        n = max(k+l-1, l+lam_1)
        while A(n) >= 0:
          n += 1
        return (n, k)
      else:
        # Otherwise, we can offset the polynomial so it has nonnegative
        # coefficients.
        s = max(k+l-1, l+lam_1)
        while not has_nonnegative_coefficients(NPolynomialQQ(A(var_N_QQ+s))):
          s += 1
        # Now we just need to check the first few values of `A`.
        for n in IntegerRange(max(k+l-1, l+lam_1), s):
          if A(n) < 0:
            return (n, k)
    # We apply the same approach to the $B_j^\lambda(k)$, noting that here
    # the leading coefficients are guaranteed to be positive.
    for j in IntegerRange(l-1):
      B = self.B(j)
      assert B.leading_coefficient() > 0
      s = l+lam_1-j
      while not has_nonnegative_coefficients(KPolynomialQQ(B(var_K_QQ+s))):
        s += 1
      for k in IntegerRange(l+lam_1-j, s):
        if B(k) < 0:
          return (j+k, k)
    # If we get to this point, we have proven that $a^\lambda(n, k)$ is always
    # nonnegative.
    return None
      
        
#
# Computes an `APolynomials` instance for the given partition, caching
# previously computed instances.
#
def a_polynomials(lam):
  global _a_polynomials_cache
  lam = Partition(lam)
  a_polys = _a_polynomials_cache.get(lam)
  if a_polys is not None:
    return a_polys
  a_polys = APolynomials(lam)
  _a_polynomials_cache[lam] = a_polys
  return a_polys
_a_polynomials_cache = {}

In [13]:
################################################################################
# Functions for checking the validity of most of the major theorems and lemmas #
# of the paper, as well as for proving Theorem 1.5.                            #
################################################################################

#
# Checks Theorem 2.2 for all $\lambda$ of size at most `l_max` and all $n$ at
# most `n_max`, returning a counterexample if found.
#
def cp_theorem_2_2_counterexample(l_max, n_max, *, a_fun=cp_a, h_fun=cp_h):
  for l in IntegerRange(l_max+1):
    for lam in Partitions(l):
      lam_1 = lam.get_part(0)
      for n in IntegerRange(l+lam_1, n_max+1):
        for k in IntegerRange(n+1):
          lhs = a_fun(lam, n, k)
          rhs = cp_a_from_h(lam, n, k, h_fun=h_fun)
          if lhs != rhs:
            return (lam, n, k)
  return None

#
# Checks Theorem 3.2 for all $\tau$ of length at most $r_max$ and $n$ at most
# `n_max`, returning a counterexample if found.
#
def cp_theorem_3_2_counterexample(r_max, n_max, *, h_fun=cp_h):
  for r in IntegerRange(r_max+1):
    for tau in ChainTypes(r):
      m = tau.multiplicity(1); tau_bar = tau.reduction()
      for n in IntegerRange(n_max+1):
        for k in IntegerRange(n+1):
          lhs = h_fun(tau, n, k)
          rhs = binomial(n+r, m) * h_fun(tau_bar, n, k)
          if lhs != rhs:
            return (tau, n, k)
  return None

#
# Checks Theorem 3.4 for all $\tau$ of length at most $r_max$ and $n$ at most
# `n_max`, returning a counterexample if found.
#
def cp_theorem_3_4_counterexample(
  r_max, n_max, *, h_fun=cp_h, kappa_fun=cp_kappa
):
  for r in IntegerRange(r_max+1):
    for tau in ChainTypes(r):
      if tau.multiplicity(1) >= 1:
        continue
      d = tau.degree(); omega = tau.derivative()
      for n in IntegerRange(n_max+1):
        for k in IntegerRange(n+1):
          lhs = h_fun(tau, n, k)
          rhs = sum(
            cp_zeta(omega_prime, omega) * kappa_fun(omega_prime, n, k)
          for omega_prime in ChainTypes(d))
          if lhs != rhs:
            return (tau, n, k)
  return None

#
# Checks Lemma 3.8 coefficient-wise for all $\tau$ of length at most $r_max$ and
# all $n$ at most `n_max`, returning a counterexample if found.
#
def cp_lemma_3_8_counterexample(r_max, n_max, *, h_fun=cp_h):
  for r in IntegerRange(r_max+1):
    for tau in ChainTypes(r):
      H = cp_H(tau)
      for n in IntegerRange(n_max+1):
        for k in IntegerRange(n+1):
          lhs = h_fun(tau, n, k)
          rhs = H[n].monomial_coefficient((var_x^(n-k)*var_y^k).polynomial())
          if lhs != rhs:
            return (tau, n, k)
  return None

#
# Checks Lemmas 3.9 and 3.10 for all $\tau$ of length at most $r_max$, returning
# a counterexample if found.
#
def cp_lemma_3_9_or_3_10_counterexample(r_max):
  for r in IntegerRange(r_max+1):
    for tau in ChainTypes(r):
      d = tau.degree(); nu = tau.rank()
      P = cp_P(tau)
      if nu == d:
        continue
      if not (2*(var_w+1)).divides(P):
        return tau
  return None

#
# Checks Lemma 4.2 for all $r$ from `r_min` to `r_max` and for $j+k$ at most
# `n_max`, returning a counterexample if found.
#
def cp_lemma_4_2_counterexample(r_min, r_max, n_max):
  for r in IntegerRange(r_min, r_max+1):
    for eps in IntegerRange(2):
      for delta in IntegerRange(2):
        gamma = eps+delta-eps*delta
        G = XYSeries(cp_Q^(-1-r) * (
          1 if (eps, delta) == (0, 0) else (
            (1+(-1)^delta*var_x+(-1)^eps*var_y)/2
          )
        ))
        for n in IntegerRange(max(0, gamma-r), n_max+1):
          for k in IntegerRange(n+1):
            j = n-k
            lhs = G[n].monomial_coefficient((var_x^j*var_y^k).polynomial())
            if r % 2 == 0 and 2*(j-eps) >= -r and 2*(k-delta) >= -r:
              r_0 = Integer(r/2)
              rhs = (
                2^(-r_0) * factorial(j+k+r_0-eps*delta) / (
                  bifactorial(2*r_0-1) *
                  factorial(j+r_0-eps) * factorial(k+r_0-delta)
                )
              ) * factorial(j+k+r-gamma) / (factorial(j) * factorial(k))
            elif r % 2 == 1 and r > 0:
              r_1 = Integer((r-1)/2)
              rhs = (
                2^(-r_1-gamma) * bifactorial(2*(j+k+r_1-eps*delta)+1) / (
                  factorial(r_1) *
                  bifactorial(2*(j+r_1-eps)+1) * bifactorial(2*(k+r_1-delta)+1)
                )
              ) * factorial(j+k+r-gamma) / (factorial(j) * factorial(k))
            else:
              rhs = 0
            if lhs != rhs:
              return (r, eps, delta, j, k)
  return None

#
# Checks Theorem 4.3 for $j+k$ at most `n_max` for `sample_count` many randomly
# generated polynomials $F$ and integers $d$, $a$, and $b$ subject to the given
# constraints, returning a counterexample if found.
#
def cp_theorem_4_3_counterexample(
  d_min, d_max, a_max, b_max, n_max, sample_count, *, coeff_max = 1000
):
  for i in range(sample_count):
    d = IntegerRange(d_min, d_max+1).random_element()
    a = IntegerRange(a_max+1).random_element()
    b = IntegerRange(b_max+1).random_element()
    F = sum(
      IntegerRange(-coeff_max, coeff_max+1).random_element() *
      var_u^alpha * var_v^beta
    for alpha in IntegerRange(a+1) for beta in IntegerRange(b+1))
    series_polys = SeriesPolynomials(F, d, a, b)
    G = series_polys.G()
    for n in IntegerRange(max(-d, 0), n_max):
      for k in IntegerRange(n+1):
        j = n-k
        lhs = G[n].monomial_coefficient((var_x^j*var_y^k).polynomial())
        rhs = series_polys.g(j, k)
        if lhs != rhs:
          return series_polys
  return None

#
# Checks Corollary 4.3.1 for $\tau$ of length at most `r_max` and $n$ at most
# `n_max`, returning a counterexample if found.
#
def cp_corollary_4_3_1_counterexample(r_max, n_max):
  for r in IntegerRange(r_max+1):
    for tau in ChainTypes(r):
      if tau.multiplicity(1) >= 1:
        continue
      d = tau.degree(); nu = tau.rank()
      H = cp_H(tau)
      h_polys = h_polynomials(tau) if nu < d else None
      for n in IntegerRange(n_max+1):
        for k in IntegerRange(n+1):
          j = n-k
          lhs = H[n].monomial_coefficient((var_x^j*var_y^k).polynomial())
          if nu == d:
            if d % 2 == 0:
              d_0 = Integer(d/2)
              rhs = (
                2^(-d_0) * factorial(n+d_0) / (
                  bifactorial(2*d_0-1) *
                  factorial(k+d_0) * factorial(n-k+d_0)
                )
              ) * factorial(n+d) / (factorial(k) * factorial(n-k))
            else:
              d_1 = Integer((d-1)/2)
              rhs = (
                2^(-d_1) * bifactorial(2*(n+d_1)+1) / (
                  factorial(d_1) *
                  bifactorial(2*(k+d_1)+1) * bifactorial(2*(n-k+d_1)+1)
                )
              ) * factorial(n+d) / (factorial(k) * factorial(n-k))
          else:
            rhs = h_polys.h(n, k)
          if lhs != rhs:
            return (tau, n, k)
  return None

#
# Checks Lemma 5.1 for all $\lambda$ of size at most `l_max`, returning a
# counterexample if found.
#
def cp_lemma_5_1_counterexample(l_max):
  for l in IntegerRange(l_max+1):
    for lam in Partitions(l):
      lam_prime = lam.derivative(); l_prime = lam_prime.size()
      for r in IntegerRange(l_prime, l+1):
        for tau in ChainTypes(r):
          m = tau.multiplicity(1)
          theta = cp_theta(tau, lam)
          if not factorial(m).divides(theta):
            return (tau, lam)
  return None

#
# Checks Theorem 1.3 for all $\lambda$ of size at most `l_max` and $n$ at most
# `n_max`, returning a counterexample if found.
#
def cp_theorem_1_3_counterexample(l_max, n_max, *, a_fun=cp_a):
  for l in IntegerRange(1, l_max+1):
    for lam in Partitions(l):
      lam_1 = lam.get_part(0)
      a_polys = a_polynomials(lam)
      # Check that the formula for $a^\lambda(n, k)$ works
      for n in IntegerRange(l+lam_1, n_max+1):
        for k in IntegerRange(n+1):
          lhs = a_fun(lam, n, k)
          rhs = a_polys.a(n, k)
          if lhs != rhs or (k < l and lhs != 0):
            return (lam, n, k)
      # Check that the leading coefficients of the polynomials $B_j^\lambda(k)$
      # and $\hat{a}_i^\lambda(n, k)$ are as predicted
      chi_val_0 = cp_chi(lam)(SymmetricGroup(l)())
      chi_val_1 = Integer(sum(
        cp_chi(mu)(SymmetricGroup(l-1)())
      for mu in Partitions(l-1, outer=lam)))
      for j in IntegerRange(l-1):
        B = a_polys.B(j)
        lhs = B.leading_coefficient()
        rhs = 2^j * chi_val_0 / (bifactorial(l-1) * bifactorial(2*j+l))
        if B.leading_coefficient() != rhs:
          return (lam, j)
      lhs_0 = a_polys.leading_coefficient_0()
      lhs_1 = 2^(1-l) * a_polys.leading_coefficient_1()
      rhs_0 = chi_val_0 if l % 2 == 0 else -chi_val_1
      rhs_1 = chi_val_0 if l % 2 == 1 else -chi_val_1
      if lhs_0 != rhs_0 or lhs_1 != rhs_1:
        return lam
  return None

#
# Checks that $a^\lambda(n, k)$ is nonnegative for all $n$ and $k$ whenever
# $\lambda$ has length at most `l_max`, returning a counterexample if found.
# Theorem 1.5 is proven by evaluating `cp_theorem_1_5_counterexample(8)` and
# observing that the result is `None`.
#
def cp_theorem_1_5_counterexample(l_max):
  for l in IntegerRange(1, l_max+1):
    for lam in Partitions(l):
      a_polys = a_polynomials(lam)
      counterexample = a_polys.positivity_counterexample()
      if counterexample is not None:
        (n, k) = counterexample
        return (lam, n, k)
  return None

#
# Returns a dictionary which associates to each partition $\lambda$ of length at
# most `l_max` a list containing the numbers of conjugate pairs of non-real
# roots for the polynomials $A_k^\lambda(n)$ with $k$ at most `k_max`. The
# comment preceding Question 7.2 from the paper is proven by inspecting the
# result of evaluating `cp_question_7_2_data(8, 40)`.
#
def cp_question_7_2_data(l_max, k_max):
  result = {}
  for l in IntegerRange(1, l_max+1):
    for lam in Partitions(l):
      a_polys = a_polynomials(lam)
      conj_pair_counts = []
      for k in IntegerRange(l, k_max+1):
        A = a_polys.A(k)
        if A == 0:
          conj_pair_counts.append(0)
        else:
          root_count = len(real_roots(a_polys.A(k)))
          conj_pair_counts.append(Integer((A.degree()-root_count)/2))
      result[lam] = conj_pair_counts
  return result

#
# Checks Conjecture 7.1 for all chain types $\tau$ of length at most `r_max`,
# returning a counterexample if found. The observation directly preceding
# Conjecture 7.1 in the paper is obtained by evaluating
# `cp_conjecture_7_1_counterexample(9)`.
#
def cp_conjecture_7_1_counterexample(r_max):
  for r in IntegerRange(r_max+1):
    for tau in ChainTypes(r):
      if not has_nonnegative_coefficients(cp_P(tau)):
        return tau
  return None

In [17]:
#
# Below is a list of example tests that can be run using the above functions.
# On the author's computer, all of them complete in under a minute in a fresh
# kernel.
#

# print("Searching for counterexamples...")
# print(f"Theorem 2.2: {cp_theorem_2_2_counterexample(5, 5)}")
# print(f"Theorem 3.2: {cp_theorem_3_2_counterexample(3, 3)}")
# print(f"Theorem 3.4: {cp_theorem_3_4_counterexample(3, 3)}")
# print(f"Lemma 3.8: {cp_lemma_3_8_counterexample(3, 3)}")
# print(f"Lemmas 3.9 and 3.10: {cp_lemma_3_9_or_3_10_counterexample(8)}")
# print(f"Lemma 4.2: {cp_lemma_4_2_counterexample(-10, 10, 10)}")
# print(f"Theorem 4.3: {cp_theorem_4_3_counterexample(-20, 20, 15, 15, 5, 100)}")
# print(f"Lemma 5.1: {cp_lemma_5_1_counterexample(5)}")
# print(f"Theorem 1.3: {cp_theorem_1_3_counterexample(5, 5)}")
# print("Done!")

In [15]:
#
# Uncomment and run the code below to verify the second component of
# Theorem 1.5. On the author's computer, it completes in around ten minutes on a
# fresh kernel and just under one minute once the `APolynomials` instances for
# for all partitions of size at mots 8 have been cached.
#

# print("Searching for counterexamples to Theorem 1.5...")
# counterexample = cp_theorem_1_5_counterexample(8)
# if counterexample is not None:
#   print(f"Counterexample found: (lam, n, k) = {counterexample}")
# else:
#   print(f"No counterexamples found")
# print("Done!")

In [16]:
#
# Uncomment and run the code below to confirm the comment preceding Question 7.2
# in the paper. On the author's computer, it completes in around ten minutes on
# a fresh kernel and just over one minute once the `APolynomials` instances for
# all partitions of size at most 8 have been cached.
#

# print("Collecting data on Question 7.2...")
# data = cp_question_7_2_data(8, 40)
# for (lam, conj_pair_counts) in data.items():
#   lam_str = f"{lam}: "
#   print(f"{lam_str:<30} {conj_pair_counts}")
# print("Done!")

In [17]:
#
# Uncomment and run the code below to verify Conjecture 7.1 for all chain types
# of length at most 9. On the author's computer, it takes less than 20 seconds
# to complete in a fresh kernel and completes almost instantaneously once the
# polynomials $P^\tau$ have been cached.
#

# print("Searching for counterexamples to Conjecture 7.1...")
# counterexample = cp_conjecture_7_1_counterexample(9)
# if counterexample is not None:
#   print(f"Counterexample found: tau = {counterexample}")
# else:
#   print(f"No counterexamples found")
# print("Done!")